In [1]:
import numpy as np
import time
import copy
import sys
sys.path.append('/home/ziniu.wzn/BayesCard')
import pandas as pd
import time
import bz2
import pickle
import logging
import ast

#from DataPrepare.join_data_preparation import JoinDataPreparator
from Models.pgmpy_BN import Pgmpy_BN

In [ ]:
data = pd.read_csv("/home/ziniu.wzn/DMV/DMV.csv")
new_cols = []
for col in data.columns:
    if col in ['VIN', 'Zip', 'City', 'Make', 'Unladen Weight', 'Maximum Gross Weight', 'Passengers', 'Reg Valid Date', 'Reg Expiration Date', 'Color']:
        data = data.drop(col, axis=1)
    else:
        new_cols.append(col.replace(" ", "_"))
data.columns = new_cols
data.head(10)

In [ ]:
tic = time.time()
BN = Pgmpy_BN('dmv')
BN.build_from_data(data, algorithm="chow-liu", max_parents=4, ignore_cols=['id'], sample_size=100000)
print(time.time()-tic)

In [2]:
import itertools
import networkx as nx
import numpy as np
import time
from tqdm import tqdm
from collections import defaultdict
from itertools import chain
import copy
from Pgmpy.inference import Inference
from Pgmpy.factors import factor_product
from Pgmpy.models import BayesianModel, JunctionTree
from Pgmpy.inference.EliminationOrder import (
    WeightedMinFill,
    MinNeighbors,
    MinFill,
    MinWeight,
)
from Pgmpy.factors.discrete import TabularCPD

class VariableElimination(object):
    def __init__(self, model, probs=None):
        model.check_model()
        self.model = model
        if probs is not None:
            self.probs = probs
        elif len(self.model.probs) != 0:
            self.probs = model.probs
        else:
            self.probs = dict()

        if isinstance(model, JunctionTree):
            self.variables = set(chain(*model.nodes()))
        else:
            self.variables = model.nodes()

        self.cardinality = {}
        self.factors = defaultdict(list)

        if isinstance(model, BayesianModel):
            self.state_names_map = {}
            for node in model.nodes():
                cpd = model.get_cpds(node)
                if isinstance(cpd, TabularCPD):
                    self.cardinality[node] = cpd.variable_card
                    cpd = cpd.to_factor()
                for var in cpd.scope():
                    self.factors[var].append(cpd)
                self.state_names_map.update(cpd.no_to_name)

        elif isinstance(model, JunctionTree):
            self.cardinality = model.get_cardinality()

            for factor in model.get_factors():
                for var in factor.variables:
                    self.factors[var].append(factor)
        self.root = self.get_root()

    def get_root(self):
        """Returns the network's root node."""

        def find_root(graph, node):
            predecessor = next(self.model.predecessors(node), None)
            if predecessor:
                root = find_root(graph, predecessor)
            else:
                root = node
            return root

        return find_root(self, list(self.model.nodes)[0])

    def steiner_tree(self, nodes):
        """Returns the minimal part of the tree that contains a set of nodes."""
        sub_nodes = set()

        def walk(node, path):
            if len(nodes) == 0:
                return

            if node in nodes:
                sub_nodes.update(path + [node])
                nodes.remove(node)

            for child in self.model.successors(node):
                walk(child, path + [node])

        walk(self.root, [])
        sub_graph = self.model.subgraph(sub_nodes)
        sub_graph.cardinalities = defaultdict(int)
        for node in sub_graph.nodes:
            sub_graph.cardinalities[node] = self.model.cardinalities[node]
        return sub_graph

    def get_probs(self, attribute, values):
        """
        Calculate Pr(attr in values) where values must be a list
        """
        factor = self.probs[attribute]
        values = [factor.get_state_no(attribute, no) for no in values]
        return np.sum(factor.values[values])


    def _get_working_factors(self, variables=[], evidence=None, return_probs=False, reduce=True):
        """
        Uses the evidence given to the query methods to modify the factors before running
        the variable elimination algorithm.
        Parameters
        ----------
        evidence: dict
            Dict of the form {variable: state}
        Returns
        -------
        dict: Modified working factors.
        """

        useful_var = copy.deepcopy(variables)
        if evidence:
            useful_var += list(evidence.keys())
        sub_graph_model = self.steiner_tree(useful_var)
        variables_sub_graph = set(sub_graph_model.nodes)

        working_factors = dict()
        for node in sub_graph_model.nodes:
            working_factors[node] = set()
            for factor in self.factors[node]:
                if set(factor.variables).issubset(variables_sub_graph):
                    working_factors[node].add((factor, None))

        if return_probs:
            probs = dict()
        # Dealing with evidence. Reducing factors over it before VE is run.
        if evidence and reduce:
            for evidence_var in evidence:
                for factor, origin in working_factors[evidence_var]:
                    factor_reduced = factor.reduce(
                        [(evidence_var, evidence[evidence_var])], inplace=False
                    )
                    if return_probs:
                        factor_reduced.normalize()
                        probs[evidence_var] = self.get_probs(evidence_var, evidence[evidence_var])
                    for var in factor_reduced.scope():
                        if var in working_factors:
                            working_factors[var].remove((factor, origin))
                            working_factors[var].add((factor_reduced, evidence_var))
                if type(evidence[evidence_var]) != list:
                    del working_factors[evidence_var]
        if return_probs:
            return working_factors, sub_graph_model, probs
        return working_factors, sub_graph_model

    def _get_elimination_order(
            self, variables=None, evidence=None, model=None, elimination_order="minfill", show_progress=False
    ):
        """
        Deals with all elimination order parameters given to _variable_elimination method
        and returns a list of variables that are to be eliminated
        Parameters
        ----------
        elimination_order: str or list
        Returns
        -------
        list: A list of variables names in the order they need to be eliminated.
        """
        if model is None:
            model = self.model
        if isinstance(model, JunctionTree):
            all_variables = set(chain(*model.nodes()))
        else:
            all_variables = model.nodes()

        if variables is None:
            to_eliminate = set(all_variables)
        else:
            not_evidence_eliminate = []
            if evidence is not None:
                for key in evidence:
                    if type(evidence[key]) != list:
                        not_evidence_eliminate.append(key)
            to_eliminate = (
                    set(all_variables)
                    - set(variables)
                    - set(not_evidence_eliminate)
            )

        # Step 1: If elimination_order is a list, verify it's correct and return.
        if hasattr(elimination_order, "__iter__") and (
                not isinstance(elimination_order, str)
        ):
            if any(
                    var in elimination_order
                    for var in set(variables).union(
                        set(evidence.keys() if evidence else [])
                    )
            ):
                raise ValueError(
                    "Elimination order contains variables which are in"
                    " variables or evidence args"
                )
            else:
                return elimination_order

        # Step 2: If elimination order is None or a Markov model, return a random order.
        elif (elimination_order is None) or (not isinstance(model, BayesianModel)):
            return to_eliminate

        # Step 3: If elimination order is a str, compute the order using the specified heuristic.
        elif isinstance(elimination_order, str) and isinstance(
                model, BayesianModel
        ):
            heuristic_dict = {
                "weightedminfill": WeightedMinFill,
                "minneighbors": MinNeighbors,
                "minweight": MinWeight,
                "minfill": MinFill,
            }
            elimination_order = heuristic_dict[elimination_order.lower()](
                model
            ).get_elimination_order(nodes=to_eliminate, show_progress=show_progress)
            return elimination_order

    def _variable_elimination(
            self,
            variables,
            operation,
            evidence=None,
            elimination_order="minfill",
            joint=True,
            show_progress=False,
    ):
        """
        Implementation of a generalized variable elimination.

        Parameters
        ----------
        variables: list, array-like
            variables that are not to be eliminated.

        operation: str ('marginalize' | 'maximize')
            The operation to do for eliminating the variable.

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: str or list (array-like)
            If str: Heuristic to use to find the elimination order.
            If array-like: The elimination order to use.
            If None: A random elimination order is used.
        """
        # Step 1: Deal with the input arguments.
        if isinstance(variables, str):
            raise TypeError("variables must be a list of strings")
        if isinstance(evidence, str):
            raise TypeError("evidence must be a list of strings")

        # Dealing with the case when variables is not provided.
        if not variables:
            all_factors = []
            for factor_li in self.factors.values():
                all_factors.extend(factor_li)
            if joint:
                return factor_product(*set(all_factors))
            else:
                return set(all_factors)

        # Step 2: Prepare data structures to run the algorithm.
        eliminated_variables = set()
        # Get working factors and elimination order
        # tic = time.time()
        working_factors, sub_graph_model = self._get_working_factors(variables, evidence)
        # toc = time.time()
        # print(f"getting working factors takes {toc-tic} secs")
        elimination_order = self._get_elimination_order(
            variables, evidence, sub_graph_model, elimination_order, show_progress=show_progress
        )
        # print(f"getting elimination orders takes {time.time()-toc} secs")
        # Step 3: Run variable elimination
        if show_progress:
            pbar = tqdm(elimination_order)
        else:
            pbar = elimination_order

        for var in pbar:
            #tic = time.time()
            # print(var)
            if show_progress:
                pbar.set_description("Eliminating: {var}".format(var=var))
            # Removing all the factors containing the variables which are
            # eliminated (as all the factors should be considered only once)
            factors = [
                factor
                for factor, _ in working_factors[var]
                if not set(factor.variables).intersection(eliminated_variables)
            ]
            phi = factor_product(*factors)
            phi = getattr(phi, operation)([var], inplace=False)
            del working_factors[var]
            for variable in phi.variables:
                if variable in working_factors:
                    working_factors[variable].add((phi, var))
            eliminated_variables.add(var)
            # print(f"eliminating {var} takes {time.time()-tic} secs")

        # Step 4: Prepare variables to be returned.
        #tic = time.time()
        final_distribution = set()
        for node in working_factors:
            for factor, origin in working_factors[node]:
                if not set(factor.variables).intersection(eliminated_variables):
                    final_distribution.add((factor, origin))
        final_distribution = [factor for factor, _ in final_distribution]
        # print(final_distribution)
        # print(f"the rest takes {time.time()-tic} secs")
        if joint:
            if isinstance(self.model, BayesianModel):
                return factor_product(*final_distribution).normalize(inplace=False)
            else:
                return factor_product(*final_distribution)
        else:
            query_var_factor = {}
            for query_var in variables:
                phi = factor_product(*final_distribution)
                query_var_factor[query_var] = phi.marginalize(
                    list(set(variables) - set([query_var])), inplace=False
                ).normalize(inplace=False)
            return query_var_factor

    def query(
            self,
            variables,
            evidence=None,
            elimination_order="weightedminfill",
            joint=True,
            show_progress=False,
    ):
        """
        Parameters
        ----------
        variables: list
            list of variables for which you want to compute the probability

        evidence: dict
            a dict key, value pair as {var: state_of_var_observed}
            None if no evidence

        elimination_order: list
            order of variable eliminations (if nothing is provided) order is
            computed automatically

        joint: boolean (default: True)
            If True, returns a Joint Distribution over `variables`.
            If False, returns a dict of distributions over each of the `variables`.
        """
        common_vars = set(evidence if evidence is not None else []).intersection(
            set(variables)
        )
        if common_vars:
            raise ValueError(
                f"Can't have the same variables in both `variables` and `evidence`. Found in both: {common_vars}"
            )

        return self._variable_elimination(
            variables=variables,
            operation="marginalize",
            evidence=evidence,
            elimination_order=elimination_order,
            joint=joint,
            show_progress=show_progress,
        )


In [3]:
with open('clt.pkl', 'rb') as f:
    BN = pickle.load(f)

In [27]:
def align_cpds_in_topological(BN):
    cpds = BN.model.cpds
    sampling_order = []
    while len(sampling_order) < len(BN.structure):
        for i, deps in enumerate(BN.structure):
            if i in sampling_order:
                continue  # already ordered
            if all(d in sampling_order for d in deps):
                sampling_order.append(i)
    topological_order = sampling_order
    topological_order_node = [BN.node_names[i] for i in sampling_order]
    new_cpds = []
    for n in topological_order_node:
        for cpd in cpds:
            if cpd.variable == n:
                new_cpds.append(cpd)
                break
    assert len(cpds) == len(new_cpds)
    return new_cpds, topological_order, topological_order_node

In [60]:
cpds[0].values.shape

(4,)

In [28]:
cpds, topological_order, topological_order_node = align_cpds_in_topological(BN)

In [140]:
def get_condition(evidence, cpd, topological_order_node, var_evidence):
    values = cpd.values
    if evidence[0][0] == -1:
        assert len(values.shape) == 1
        probs = values[var_evidence]
        return_prob = np.sum(probs)
        probs = probs/return_prob  #re-normalize
        new_evidence = np.random.choice(var_evidence, p=probs, size=evidence.shape[-1])
    else:
        scope = cpd.variable
        scope_ind = topological_order_node.index(scope)
        condition = cpd.variables[1:]
        condition_ind = [topological_order_node.index(c) for c in condition]
        condition_evidence = evidence[condition_ind]
        #the following is hardcoded for fast computation
        if len(condition) == 1:
            probs = values[:,condition_evidence[0]]
        elif len(condition) == 2:
            probs = values[:,condition_evidence[0], condition_evidence[1]]
        elif len(condition) == 3:
            probs = values[:,condition_evidence[0], condition_evidence[1], condition_evidence[2]]
        elif len(condition) == 4:
            probs = values[:,condition_evidence[0], condition_evidence[1], condition_evidence[2],
                              condition_evidence[3]]
        elif len(condition) == 5:
            probs = values[:,condition_evidence[0], condition_evidence[1], condition_evidence[2],
                              condition_evidence[3], condition_evidence[4]]
        elif len(condition) == 6:
            probs = values[:,condition_evidence[0], condition_evidence[1], condition_evidence[2],
                              condition_evidence[3], condition_evidence[4], condition_evidence[5]]
        elif len(condition) == 7:
            probs = values[:,condition_evidence[0], condition_evidence[1], condition_evidence[2],
                              condition_evidence[3], condition_evidence[4], condition_evidence[5], 
                               condition_evidence[6]]
        else:
            #no more efficient tricks
            probs = np.zeros((values.shape[0], evidence.shape[-1]))
            for j in range(values.shape[0]):
                probs[j,:] = values[j]
        
        probs = probs[var_evidence,:]
        return_prob = np.sum(probs, axis=0)
        probs = (probs/return_prob)
        probs[np.isnan(probs)]=0
        generate_probs = probs.mean(axis=1)
        generate_probs = generate_probs/np.sum(generate_probs)
        new_evidence = np.random.choice(var_evidence, p=generate_probs, size=evidence.shape[-1])
        #np.asarray([np.random.choice(var_evidence, p=probs[:,i]) for i in range(evidence.shape[-1])])
    return return_prob, new_evidence
    
    
def progressive_sampling(query, cpds, topological_order, topological_order_node, sample_size):
    evidence = np.zeros((len(topological_order), sample_size), dtype=int)-1
    probs = np.ones(sample_size)
    for i, node in enumerate(topological_order_node):
        if node in query:
            var_evidence = query[node]
        else:
            var_evidence = np.arange(cpds[i].values.shape[0])
        new_probs, new_evidence = get_condition(evidence, cpds[i], topological_order_node, var_evidence)
        evidence[i,:] = new_evidence
        probs *= new_probs
    return np.sum(probs)/evidence.shape[-1]

In [8]:
BN.infer_algo = "progressive_sampling"
BN.init_inference_method()
with open("../Benchmark/DMV/query.sql") as f:
    queries = f.readlines()
from Evaluation.cardinality_estimation import parse_query_single_table

In [10]:
query_str = queries[0].split("||")[0]
query_str = parse_query_single_table(query_str.strip(), BN)
#query_str, n_distinct = BN.query_decoding(query_str, None)
tic = time.time()
a = BN.query(query_str, sample_size=1000)
print(time.time()-tic)
print(a)

0.0038089752197265625
1473.2108010077902


In [103]:
BN.init_inference_method()

In [11]:
with open("../Benchmark/DMV/query.sql") as f:
    queries = f.readlines()
res = []
cards = []
lat = []
for q in queries:
    query_str = q.split("||")[0]
    card = int(q.split("||")[-1])
    cards.append(card)
    query_str = parse_query_single_table(query_str.strip(), BN)
    #print(BN.query(query_str))
    #query_str, n_distinct = BN.query_decoding(query_str, None)
    tic=time.time()
    a = BN.query(query_str, sample_size=1000)
    lat.append(time.time()-tic)
    res.append(a)

In [12]:
print(f"average latency: {int(np.mean(lat)*100000)/100}ms")
pred = np.asarray(res)
pred[pred<=1.0] = 1
cards = np.asarray(cards)
errors = np.maximum(np.divide(pred, cards), np.divide(pred, cards))
for i in [50,90,95,99,100]:
    print(f"{i}% quantile: {np.percentile(errors, i)}")

average latency: 2.47ms
50% quantile: 1.0011005240041022
90% quantile: 1.194691158063247
95% quantile: 1.4821890581670953
99% quantile: 2.6233488737497614
100% quantile: 7.073564074884702


In [151]:
print(f"average latency: {int(np.mean(lat)*100000)/100}ms")
pred = np.asarray(res)
pred[pred<=1.0] = 1
cards = np.asarray(cards)
errors = np.maximum(np.divide(pred, cards), np.divide(pred, cards))
for i in [50,90,95,99,100]:
    print(f"{i}% quantile: {np.percentile(errors, i)}")

average latency: 8.11ms
50% quantile: 1.0006913583865398
90% quantile: 1.1824712302488722
95% quantile: 1.4575186136351088
99% quantile: 2.5862646434652756
100% quantile: 7.0735871324767


In [74]:
n = 1000
s = 1000
var_evidence = np.arange(n)
probs = np.ones((n,s))
probs = probs/np.sum(probs, axis=0)
tic = time.time()
a = np.asarray([np.random.choice(var_evidence, p=probs[:,i]) for i in range(s)])
print(time.time()-tic)

0.036821842193603516


In [75]:
a

array([910, 260, 757, 507, 467, 478, 193, 662, 351, 229, 472, 705, 660,
       873, 712, 119,  96, 441,  54, 556, 161, 101, 994, 479, 781, 173,
        36, 653, 899, 937,  73, 808, 977, 376, 431, 366, 283, 109, 795,
       280, 394, 661,  73, 508, 540, 473, 925, 708, 761, 197, 120, 966,
       937, 107, 763, 701, 844, 400, 582, 474, 961, 971, 810, 469, 602,
       338, 455, 308, 194, 946, 133, 789, 763, 526, 430, 499, 633,  92,
       213, 909, 705, 402, 763, 422, 731, 990, 653, 920, 343, 840, 302,
       216, 596, 762,  72, 639, 345, 488, 218, 655, 427, 464, 833, 862,
       862, 339, 343, 374, 208,  67, 837, 679, 296, 949, 766, 177, 988,
       797, 189, 618, 855, 456, 391, 328, 710, 368, 635, 510, 757, 210,
       311, 832, 226, 645, 401, 795, 533, 690, 969, 932,  88, 586, 359,
       931, 240, 589, 588, 671,   3, 531, 321, 504, 705, 530, 986,  63,
       303, 969, 939, 511, 326,  47, 189, 680, 495, 207, 510, 573, 555,
       115, 732, 185, 332, 742, 263, 664, 454, 150, 976, 556, 80